# Using EcoFOCIpy to process raw field data

## Cruise ID - DY2407

## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

In [1]:
import yaml
import glob
import pandas as pd

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2024/CTDcasts/dy2407/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
nutdatafile = sample_data_dir+'working/DiscreteNutrients/DY2407 Nutrient Data.txt' #<- point to cruise and process all files within
cruise_name = 'dy2407' #no hyphens
cruise_meta_file = sample_data_dir+'logs/dy2407.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpNutsWOCE.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2407/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2407/rawconverted/ctd002.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [4]:
# for SKQ only - drop lat/lon from datafile in lew of metadata source
# cruise_data = {}
# for cast,cdata in cruise_data_temp.items():
#     cdata.drop(labels=['latitude','longitude'],axis=1,inplace=True)
#     cruise_data.update({cast:cdata})

## Load csv Nutrient File

In [5]:
nut_data = pd.read_csv(nutdatafile,delimiter='\t')
nut_data

,Cast,Niskin,PO4 (uM),PO4_Flag,Sil (uM),Sil_Flag,NO3 (uM),NO3_Flag,NO2 (uM),NO2_Flag,NH4 (uM),NH4_Flag
0,1,7,0.5320,2,11.40,6,0.44,2,0.06,2,0.55,2
1,1,6,0.7340,6,12.30,2,2.15,6,0.12,6,0.75,6
2,1,5,0.8640,2,13.35,6,4.35,2,0.15,2,0.91,2
3,1,4,0.8900,2,14.50,2,5.28,2,0.12,2,0.66,2
4,1,3,1.3350,2,22.10,6,10.95,2,0.15,2,2.59,2
5,1,2,1.4315,6,24.20,2,12.20,6,0.18,6,2.15,6
6,1,1,1.3710,2,24.10,2,12.05,2,0.15,2,2.11,2
7,2,1,1.4300,6,20.60,2,10.20,6,0.13,6,1.35,6
8,2,2,1.4640,2,24.70,6,12.14,2,0.16,2,1.24,2
9,2,3,1.4485,6,25.90,2,12.65,6,0.15,6,1.42,6


## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [6]:
keep_param = ['bottle','prdm']
# keep_param = ['bottle','prsm']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(nut_data[nut_data['Cast']==matchcast],cdata.reset_index()[keep_param],right_on='bottle',left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
    except:
        continue

## Add Deployment meta information

In [7]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)

## Add institutional meta-information


In [9]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [10]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'Sil (uM)':'SI',
                            'PO4 (uM)':'PO4',
                            'NO2 (uM)':'NO2', 
                            'NO3 (uM)':'NO3',
                            'NH4 (uM)':'NH4',
                            'Sil_Flag':'SI_WOCE_FLAG',
                            'PO4_Flag':'PO4_WOCE_FLAG',
                            'NO2_Flag':'NO2_WOCE_FLAG', 
                            'NO3_Flag':'NO3_WOCE_FLAG',
                            'NH4_Flag':'NH4_WOCE_FLAG',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history